In [1]:
import open3d as o3d
import numpy as np
import torch
import os
import json

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [11]:
import os
import json
import numpy as np
import open3d as o3d

def process_point_clouds(dataset_dir, save_dir):
    """
    Process point clouds, create masks, and generate summary files for each scene.
    
    Args:
    dataset_dir (str): Path to the dataset directory.
    save_dir (str): Path to save the processed results.
    
    Returns:
    tuple: Dictionaries of unique classes and parts (cls_dict, part_dict).
    """
    scene_dir = os.path.join(dataset_dir, 'part_valid')
    gt_instance_dir = os.path.join(dataset_dir, 'mask_valid_gt')
    id2cls_dir = os.path.join(dataset_dir,'id2ins_valid_gt')
    id2part_dir = os.path.join(dataset_dir,'id2part_valid_gt')
    # Ensure the save directory exists
    os.makedirs(save_dir, exist_ok=True)

    cls_dict = {}
    part_dict = {}
    cls_index = 0
    part_index = 0

    for scene_id in os.listdir(scene_dir)[:50]:
        # Create a directory for each scene
        scene_save_dir = os.path.join(save_dir, scene_id)
        os.makedirs(scene_save_dir, exist_ok=True)
        
        # Create pred_mask directory within the scene directory
        pred_mask_dir = os.path.join(scene_save_dir, 'pred_mask')
        os.makedirs(pred_mask_dir, exist_ok=True)

        # Load JSON files
        with open(os.path.join(id2cls_dir, f'id2ins_{scene_id}.json'), 'r') as f:
            id2cls = json.load(f)
        with open(os.path.join(id2part_dir, f'id2part_r{scene_id}.json'), 'r') as f:
            id2part = json.load(f)

        # Process unique classes and parts
        for cls in set(id2cls.values()):
            if cls not in cls_dict:
                cls_dict[cls] = cls_index
                cls_index += 1
        for part in set(id2part.values()):
            if part not in part_dict:
                part_dict[part] = part_index
                part_index += 1

        # Read the point cloud
        scene_ply = o3d.io.read_point_cloud(os.path.join(scene_dir, scene_id, f'points_{scene_id}.ply'))
        points = np.asarray(scene_ply.points)

        # Load the mask
        masks = np.loadtxt(os.path.join(gt_instance_dir, f'mask_{scene_id}.txt'))

        # Ensure masks have the same length as points
        assert len(masks) == len(points), f"Mismatch in length for scene {scene_id}"

        # Get unique labels
        unique_labels = np.unique(masks)

        # Prepare summary file
        summary_file_path = os.path.join(scene_save_dir, f'{scene_id}_summary.txt')
        with open(summary_file_path, 'w') as summary_file:
            # Process each unique label
            for label in unique_labels:
                if label == 0:  # Assuming 0 is background or unlabeled
                    continue

                # Extract points for this label
                label_mask = masks == label
                label_points = points[label_mask]

                # Save the mask for this label
                mask_filename = f'{int(label):03d}.txt'
                mask_filepath = os.path.join(pred_mask_dir, mask_filename)
                np.savetxt(mask_filepath, label_mask.astype(int), fmt='%d')

                # Get class information
                class_name = id2cls.get(str(int(label)), "unknown")
                class_index = cls_dict.get(class_name, -1)

                # Calculate confidence (placeholder, you may want to implement your own logic)
                confidence = 1.0

                # Write to summary file
                summary_file.write(f"pred_mask/{mask_filename} {class_index} {confidence}\n")

        print(f"Processed scene {scene_id}")

    print("All scenes processed.")
    return cls_dict, part_dict

# Example usage
if __name__ == "__main__":
    dataset_dir = '/home/wan/Datasets/Test_scene'
    save_dir = 'part_scene_results'
    cls_dict, part_dict = process_point_clouds(dataset_dir, save_dir)

    # Optionally, save cls_dict and part_dict
    with open('cls_dict.json', 'w') as f:
        json.dump(cls_dict, f, indent=2)
    with open('part_dict.json', 'w') as f:
        json.dump(part_dict, f, indent=2)

    print("Unique classes:")
    for cls, index in cls_dict.items():
        print(f"{index}: {cls}")

    print("\nUnique parts:")
    for part, index in part_dict.items():
        print(f"{index}: {part}")

Processed scene 0063
Processed scene 0051
Processed scene 0228
Processed scene 0275
Processed scene 0041
Processed scene 0022
Processed scene 0199
Processed scene 0271
Processed scene 0152
Processed scene 0113
Processed scene 0014
Processed scene 0054
Processed scene 0052
Processed scene 0064
Processed scene 0053
Processed scene 0082
Processed scene 0240
Processed scene 0176
Processed scene 0097
Processed scene 0069
Processed scene 0191
Processed scene 0159
Processed scene 0232
Processed scene 0150
Processed scene 0104
Processed scene 0083
Processed scene 0246
Processed scene 0025
Processed scene 0230
Processed scene 0248
Processed scene 0024
Processed scene 0143
Processed scene 0149
Processed scene 0231
Processed scene 0220
Processed scene 0103
Processed scene 0233
Processed scene 0283
Processed scene 0267
Processed scene 0288
Processed scene 0045
Processed scene 0100
Processed scene 0004
Processed scene 0135
Processed scene 0209
Processed scene 0081
Processed scene 0251
Processed sce